In [1]:
!pip install dash jupyter-dash pandas


In [2]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
from jupyter_dash import JupyterDash  # Pour exécuter Dash dans Jupyter


In [3]:
# Charger les données du CSV
def load_data():
    try:
        df = pd.read_csv("eth_prices.csv")
        df["timestamp"] = pd.to_datetime(df["timestamp"])  # Convertir en datetime
        return df
    except Exception as e:
        print(f"Erreur lors du chargement des données : {e}")
        return pd.DataFrame(columns=["timestamp", "price"])

In [4]:
# Initialisation de l'application Dash dans Jupyter
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("Suivi du prix de l'ETH", style={"textAlign": "center"}),

    # Affichage du prix actuel
    html.Div(id="latest-price", style={"fontSize": "24px", "textAlign": "center"}),

    # Graphique des prix
    dcc.Graph(id="price-graph"),

    # Rafraîchissement automatique toutes les 5 minutes
    dcc.Interval(id="interval-update", interval=5 * 60 * 1000, n_intervals=0)
])

# Callback pour mettre à jour les données
@app.callback(
    [Output("latest-price", "children"), Output("price-graph", "figure")],
    Input("interval-update", "n_intervals")
)
def update_dashboard(n):
    df = load_data()

    if df.empty:
        return "Il n'ya pas de données disponibles", {}

    latest_price = df.iloc[-1]["price"]
    latest_time = df.iloc[-1]["timestamp"]

    return (f" Dernier prix ETH : {latest_price} USDT (mis à jour à {latest_time})", 
            {
                "data": [{"x": df["timestamp"], "y": df["price"], "type": "line", "name": "Prix ETH"}],
                "layout": {"title": "Évolution du prix de l'ETH"}
            })



C:\Users\elidu\anaconda3\lib\site-packages\dash\dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [5]:
# Lancer l'application Dash dans Jupyter
app.run_server()

Dash app running on http://127.0.0.1:8050/
